<a href="https://colab.research.google.com/github/SeokcheonMoon/study_AIs/blob/main/docs/quests/MLs/SpineSurgeryList_GridSearchCV_resampling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 데이터

In [3]:
import pandas as pd
df_SSL = pd.read_csv("SpineSurgeryList.csv")
df_SSL[:2]

,Unnamed: 0,환자ID,Large Lymphocyte,Location of herniation,ODI,가족력,간질성폐질환,고혈압여부,과거수술횟수,당뇨여부,...,Modic change,PI,PT,Seg Angle(raw),Vaccum disc,골밀도,디스크단면적,디스크위치,척추이동척도,척추전방위증
0,0,1PT,22.8,3,51.0,0.0,0,0,0,0,...,3,51.6,36.6,14.4,0,-1.01,2048.5,4,Down,0
1,1,2PT,44.9,4,26.0,0.0,0,0,0,0,...,0,40.8,7.2,17.8,0,-1.14,1753.1,4,Up,0


In [4]:
df_SSL_extract = df_SSL[['성별','신장','체중', '흡연여부', '연령', '혈액형', '직업', '재발여부' ]]
df_SSL_extract

,성별,신장,체중,흡연여부,연령,혈액형,직업,재발여부
0,2,163,60.3,0,66,RH+A,자영업,0
1,1,171,71.7,0,47,RH+A,운동선수,0
2,1,178,77.1,0,39,RH+B,특수전문직,0
3,1,174,74.2,0,40,RH+O,주부,0
4,1,183,80.7,0,42,RH+A,특수전문직,0
...,...,...,...,...,...,...,...,...
1889,2,157,64.0,0,59,RH+A,무직,0
1890,2,157,59.0,0,42,RH+B,사무직,0
1891,1,167,70.0,0,61,RH+O,무직,0
1892,1,177,77.0,0,29,RH+A,사무직,0


## 전처리

In [5]:
df_SSL_extract.dropna(inplace = True)

<ipython-input-5-0f8e88518e15>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_SSL_extract.dropna(inplace = True)


In [6]:
df_SSL_extract.isnull().sum()

성별      0
신장      0
체중      0
흡연여부    0
연령      0
혈액형     0
직업      0
재발여부    0
dtype: int64

In [7]:
# target과 feature를 설정
target = df_SSL_extract["재발여부"]         # y값
pre_X = df_SSL_extract.drop(columns = ["재발여부"])         # 재발여부 컬럼만 삭제한 데이터프레임
features = pd.get_dummies(pre_X)          # onehot encoding처럼 pd.get_dummies(x값)으로 값을 뽑을 수 있다. but 나중에는 사용하지 않기
features

,성별,신장,체중,흡연여부,연령,혈액형_RH+A,혈액형_RH+AB,혈액형_RH+B,혈액형_RH+O,직업_건설업,...,직업_사무직,직업_사업가,직업_예술가,직업_운동선수,직업_운수업,직업_의료직,직업_자영업,직업_주부,직업_특수전문직,직업_학생
0,2,163,60.3,0,66,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,1,171,71.7,0,47,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,1,178,77.1,0,39,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
3,1,174,74.2,0,40,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
4,1,183,80.7,0,42,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1889,2,157,64.0,0,59,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1890,2,157,59.0,0,42,0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,0
1891,1,167,70.0,0,61,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1892,1,177,77.0,0,29,1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


## 특성공학(Feature Engineering) - Cross Validation

In [10]:
# model : hyper parameter 적용    # dataset : 알맞게 섞어서 대입
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier


hyper_parameters = {'max_depth':range(2, 10),
              'min_samples_leaf': range(2, 10),
              'criterion': ['gini','entropy'],
              'class_weight': [None, 'balanced'],
              'min_samples_split': range(2, 10)}
estimator_model = DecisionTreeClassifier()
# grid_model = GridSearchCV(estimator_model,hyper_parameters)

# score 방식 지정
from sklearn.metrics import f1_score, make_scorer
scoring = make_scorer(f1_score)

grid_model = GridSearchCV(estimator_model,hyper_parameters, scoring=scoring, cv = 5)          # cv는 데이터 셔플을 의미

grid_model

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(),
             param_grid={'class_weight': [None, 'balanced'],
                         'criterion': ['gini', 'entropy'],
                         'max_depth': range(2, 10),
                         'min_samples_leaf': range(2, 10),
                         'min_samples_split': range(2, 10)},
             scoring=make_scorer(f1_score))

## 모델

In [11]:
grid_model.fit(features,target)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(),
             param_grid={'class_weight': [None, 'balanced'],
                         'criterion': ['gini', 'entropy'],
                         'max_depth': range(2, 10),
                         'min_samples_leaf': range(2, 10),
                         'min_samples_split': range(2, 10)},
             scoring=make_scorer(f1_score))

In [12]:
grid_model.best_estimator_

DecisionTreeClassifier(class_weight='balanced', criterion='entropy',
                       max_depth=9, min_samples_leaf=8, min_samples_split=5)

In [13]:
grid_model.best_params_

{'class_weight': 'balanced',
 'criterion': 'entropy',
 'max_depth': 9,
 'min_samples_leaf': 8,
 'min_samples_split': 5}

In [14]:
grid_model.scorer_

make_scorer(f1_score)

In [15]:
best_model = grid_model.best_estimator_

## 평가

In [16]:
from sklearn.metrics import classification_report
target_predict = best_model.predict(features)
print(classification_report(target, target_predict))

              precision    recall  f1-score   support

           0       0.99      0.62      0.76      1302
           1       0.25      0.93      0.40       177

    accuracy                           0.66      1479
   macro avg       0.62      0.78      0.58      1479
weighted avg       0.90      0.66      0.72      1479



In [17]:
target.value_counts()

0    1302
1     177
Name: 재발여부, dtype: int64

## Resampling

In [19]:
# 데이터가 불균형한지 확인

from collections import Counter
Counter(target)                   # 0은 50개, 1은 20개 를 의미

Counter({0: 1302, 1: 177})

### oversampling

In [20]:
from imblearn.over_sampling import SMOTEN

oversampling = SMOTEN(sampling_strategy=0.8)
feature_oversampling, target_oversampling = oversampling.fit_resample(features,target)
feature_oversampling.shape , target_oversampling.shape

((2343, 26), (2343,))

In [21]:
Counter(target_oversampling)

Counter({0: 1302, 1: 1041})

### undersampling

In [22]:
from imblearn.under_sampling import NearMiss

undersampling = NearMiss(sampling_strategy=0.8)
feature_undersampling, target_undersampling = undersampling.fit_resample(features,target)
feature_undersampling.shape, target_undersampling.shape

((398, 26), (398,))

In [23]:
Counter(target_undersampling)

Counter({0: 221, 1: 177})

### combinesampling

In [24]:
from imblearn.combine import SMOTEENN
combinesampling = SMOTEENN(sampling_strategy=0.5)
feature_combinesampling, target_combinesampling= combinesampling.fit_resample(features,target)
feature_combinesampling.shape, target_combinesampling.shape

((1164, 26), (1164,))

In [25]:
Counter(target_combinesampling)

Counter({0: 779, 1: 385})

### Sampling data 평가

In [31]:
oversample_model = best_model.fit(feature_oversampling, target_oversampling)
target_predict_over = oversample_model.predict(features)
print(classification_report(target, target_predict_over))

              precision    recall  f1-score   support

           0       0.92      0.94      0.93      1302
           1       0.49      0.42      0.45       177

    accuracy                           0.88      1479
   macro avg       0.71      0.68      0.69      1479
weighted avg       0.87      0.88      0.87      1479



In [32]:
undersample_model = best_model.fit(feature_undersampling, target_undersampling)
target_predict_under = undersample_model.predict(features)
print(classification_report(target, target_predict_under))

              precision    recall  f1-score   support

           0       0.93      0.39      0.55      1302
           1       0.15      0.79      0.25       177

    accuracy                           0.44      1479
   macro avg       0.54      0.59      0.40      1479
weighted avg       0.84      0.44      0.51      1479



In [33]:
combinesample_model = best_model.fit(feature_combinesampling, target_combinesampling)
target_predict_combine = combinesample_model.predict(features)
print(classification_report(target, target_predict_combine))

              precision    recall  f1-score   support

           0       0.92      0.87      0.89      1302
           1       0.33      0.48      0.39       177

    accuracy                           0.82      1479
   macro avg       0.63      0.67      0.64      1479
weighted avg       0.85      0.82      0.83      1479

